# Ejercicio 47
La mediana de cantidad de links internos que tienen todos los contenidos que existen.

In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
dfPageLinks = pd.read_csv('./WikipediaDataset/pagelinks_sample.csv', usecols = ['pl_from'])
dfContents = pd.read_csv('./WikipediaDataset/contents.csv', usecols = ["id"])

## Limpiando y preparando los datasets
Se procede a limpiar los datasets para usarlos. Al ser documentos con gran cantidad de datos, se leyeron de antemano solo las columnas imprescindibles para realizar la consigna, necesario para ahorrar memoria y aumentar performance.
Se trabaja sobre la columna de ids de ambos documentos. Se eliminan posibles filas que tengan NaN.

In [2]:
dfPageLinks.head()

,pl_from
0,4840492
1,950948
2,6418187
3,3436623
4,4713066


In [3]:
dfPageLinks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 80000000 entries, 0 to 79999999
Data columns (total 1 columns):
 #   Column   Dtype
---  ------   -----
 0   pl_from  int64
dtypes: int64(1)
memory usage: 610.4 MB


In [ ]:
dfPageLinks = dfPageLinks[dfPageLinks['pl_from'].notna()]

In [ ]:
dfPageLinks["pl_from"].value_counts()

Se añaden al dataframe la cantidad de links internos que tiene el respectivo id. 

In [ ]:
dfPageLinks["cant_links_internos"] = dfPageLinks.groupby("pl_from")["pl_from"].transform('count')
dfPageLinks

Para facilitar la forma de interpretar el merge se cambia el nombre de "pl_from" a "id_fuente"

In [ ]:
dfPageLinks.rename(columns = {'pl_from': 'id_fuente'}, inplace = True)
dfPageLinks.head()

Como se va a hacer un merge con el otro dataframe, es importante eliminar las filas con ids duplicados. De esta forma, a la hora de hacer el right merge se van a obtener el número de filas del right dataframe, este es el df de contents.

In [ ]:
dfPageLinks.drop_duplicates(subset = ["id_fuente"], inplace = True, ignore_index = True)
dfPageLinks

In [ ]:
dfContents.head()

In [ ]:
dfContents.info()

In [ ]:
dfContents = dfContents[dfContents['id'].notna()]

También se cambia el nombre de la columna "id" a "id_fuente".

In [ ]:
dfContents.rename(columns = {'id': 'id_fuente'}, inplace = True)
dfContents.head()

## Desarrollo

In [ ]:
dfMerge = pd.merge(dfPageLinks, dfContents, how = "right", on = ["id_fuente"])
dfMerge

In [ ]:
dfMerge.info()

Después del merge, el dataframe resultante tiene todas las filas del dataframe contents ya que fue un right join. Sin embargo, hay ids del contents que no estaban en la lista de ids del page_links, quedando así columnas de cant_links_internos con NaN. Esto significa que los ids con NaN en cant_links_internos no tienen justamente link internos, por lo que se puede reemplazar con 0s.  

In [ ]:
dfMerge['cant_links_internos'] = dfMerge['cant_links_internos'].fillna(0)
dfMerge

In [ ]:
dfMerge["cant_links_internos"].value_counts()

Ahora se puede calcular la mediana de la columna cant_links_internos, respondiendo así la consigna.

In [ ]:
mediana = dfMerge["cant_links_internos"].median()
mediana

## Conclusiones
Al asumir que todos los contenidos que existen están en contents, y todos los links internos que existen están en page_links_sample, se pudo concluir que la mediana de links internos es 1. Esto sifnifica que el 50 % de cantidad de links internos está entre 0 y 1 y el otro 50 % es mayor o igual a 1. 